In [128]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [165]:
G = nx.gnm_random_graph(15000, 60000)

# nx.draw(G, with_labels=True)

In [105]:
def get_d(degree: list[list]|list[tuple]|nx.classes.reportviews.DegreeView) -> list:
    '''Get a list of degrees from a (converted) DegreeView'''
    return [d for v, d in degree]

def get_v(degree: list[list]|list[tuple]|nx.classes.reportviews.DegreeView) -> list:
    '''Get a list of verteces from a (converted) DegreeView'''
    return [v for v, d in degree]

In [136]:
import random

def random_sd(G: nx.Graph, k):
    return [random.randint(0, G.number_of_nodes()-1) for _ in range(k)]

### SingleDiscount

In [ ]:
def SD(G: nx.Graph, k: int) -> list[int]:
    seed = []

    dd = [list(d) for d in G.degree()]

    for _ in range(k):
        u = dd.pop(np.argmax(get_d(dd)))[0]
        seed.append(u)
        for v in G.neighbors(u):
            if v in seed:
                break
            for w in G.neighbors(v):
                if w in seed:
                    dd[get_v(dd).index(v)][1] -= 1

    return seed
k = 50
SD(G, k)

### DegreeDiscountIC

In [ ]:
p = 0.01

def DDIC(G: nx.Graph, k: int) -> list[int]:
    seed = []
    t = np.zeros(len(G))

    dd = [list(d) for d in G.degree()]

    for _ in range(k):
        u = dd.pop(np.argmax(get_d(dd)))[0]
        seed.append(u)
        for v in G.neighbors(u):
            if v in seed:
                break
            t[v] += 1
            d_v = G.degree(v)
            dd[get_v(dd).index(v)][1] = d_v - 2*t[v] - (d_v - t[v])*t[v]*p

    return seed

k = 50

DDIC(G, k)

In [154]:
# source: https://hautahi.com/ic_comparison


def propagate_nx(g,p,new_active):

    targets = []
    for node in new_active:
        targets += g.neighbors(node)

    return(targets)

def IC(graph_object,S,p,mc):
    """
    Inputs: graph_object: 4 possible network representations
                - igraph object
                - Networkx object
                - E x 2 Pandas dataframe of directed edges. Columns: ['source','target']
                - dictionary with key=source node & values=out-neighbors
            S:  List of seed nodes
            p:  Disease propagation probability
            mc: Number of Monte-Carlo simulations,
    Output: Average number of nodes influenced by seed nodes in S
    """

    # Loop over the Monte-Carlo Simulations
    spread = []
    for i in range(mc):

        # Simulate propagation process
        new_active, A = S[:], S[:]
        while new_active:

            # 1. Find out-neighbors for each newly active node
            targets = propagate_nx(graph_object,p,new_active)


            # 2. Determine newly activated neighbors (set seed and sort for consistency)
            np.random.seed(i)
            success = np.random.uniform(0,1,len(targets)) < p
            new_ones = list(np.extract(success, sorted(targets)))

            # 3. Find newly activated nodes and add to the set of activated nodes
            new_active = list(set(new_ones) - set(A))
            A += new_active

        spread.append(len(A))

    return(np.mean(spread),A)

In [ ]:
mc = 20

data = []
for k in range(1,50):
    data.append(IC(G, DDIC(G, k), p, mc)[0])
plt.plot(data)

data = []
for k in range(1,50):
    data.append(IC(G, SD(G, k), p, mc)[0])
plt.plot(data)

data = []
for k in range(1,50):
    data.append(IC(G, random_sd(G, k), p, mc)[0])
plt.plot(data)
